In [ ]:
import cv2
import numpy as np

In [ ]:
CAM_INDEX = 0
FRAME_WIDTH = 640
FRAME_HEIGHT = 480

In [ ]:
VLINE_X = 450

In [ ]:
SAFE_THRESH = 150
DANGER_THRESH = 50

In [ ]:
back_sub = cv2.createBackgroundSubtractorMOG2(history=200, varThreshold=50, detectShadows=False)

In [ ]:
cap = cv2.VideoCapture(CAM_INDEX)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

In [ ]:
def get_hand_mask(frame_bgr):
    # Optional: background subtraction to isolate moving object
    fg_mask = back_sub.apply(frame_bgr)
    fg_mask = cv2.erode(fg_mask, None, iterations=1)
    fg_mask = cv2.dilate(fg_mask, None, iterations=2)

    # Skin-color segmentation in HSV
    hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)
    lower = np.array([0, 30, 60], dtype="uint8")   # tune for your skin
    upper = np.array([20, 150, 255], dtype="uint8")
    skin_mask = cv2.inRange(hsv, lower, upper)

    # Combine background-foreground and skin mask
    mask = cv2.bitwise_and(skin_mask, fg_mask)
    mask = cv2.GaussianBlur(mask, (5, 5), 0)
    _, mask_bin = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Clean up
    kernel = np.ones((3, 3), np.uint8)
    mask_bin = cv2.morphologyEx(mask_bin, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask_bin = cv2.morphologyEx(mask_bin, cv2.MORPH_OPEN, kernel, iterations=1)

    return mask_bin

In [ ]:
def find_hand_contour(mask_bin):
    contours, _ = cv2.findContours(mask_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return None
    # Largest contour assumed to be hand
    max_cont = max(contours, key=cv2.contourArea)
    if cv2.contourArea(max_cont) < 2000:  # reject tiny blobs
        return None
    return max_cont

In [ ]:
def get_fingertip_candidate(contour):
    # Convex hull and extreme points
    hull = cv2.convexHull(contour)
    # Find top-most point (min y) as a rough fingertip
    topmost = tuple(hull[hull[:, :, 1].argmin()][0])
    return topmost, hull

In [ ]:
def distance_point_to_vertical_line(point, line_x):
    px, _ = point
    return abs(px - line_x)

In [ ]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    mask = get_hand_mask(frame)
    hand_cont = find_hand_contour(mask)

    interaction_point = None
    hull = None
    state = "NO HAND"
    color = (255, 255, 255)

    if hand_cont is not None:
        interaction_point, hull = get_fingertip_candidate(hand_cont)

        # Compute distance from fingertip to virtual line
        dist = distance_point_to_vertical_line(interaction_point, VLINE_X)

        if dist > SAFE_THRESH:
            state = "SAFE"
            color = (0, 255, 0)
        elif dist > DANGER_THRESH:
            state = "WARNING"
            color = (0, 255, 255)
        else:
            state = "DANGER"
            color = (0, 0, 255)

        # Draw hand contour + hull + fingertip
        cv2.drawContours(frame, [hand_cont], -1, (255, 0, 0), 2)
        if hull is not None:
            cv2.drawContours(frame, [hull], -1, (0, 0, 255), 2)
        if interaction_point is not None:
            cv2.circle(frame, interaction_point, 8, (255, 255, 0), -1)

        # Visualize distance
        cv2.line(frame, interaction_point, (VLINE_X, interaction_point[1]), (200, 200, 200), 1)
        cv2.putText(frame, f"d={dist:.0f}px", (interaction_point[0]+10, interaction_point[1]-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1)

    # Draw virtual boundary
    cv2.line(frame, (VLINE_X, 0), (VLINE_X, FRAME_HEIGHT), color, 3)

    # State text
    cv2.putText(frame, f"STATE: {state}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, color, 2)

    if state == "DANGER":
        cv2.putText(frame, "DANGER DANGER", (100, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)

    cv2.imshow("Hand Safety Demo", frame)
    cv2.imshow("Hand Mask", mask)

    key = cv2.waitKey(1) & 0xFF
    if key == 27 or key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()